In [1]:
!pip install pyyaml h5py 

# Setup

In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import odeint
from mpl_toolkits.mplot3d import Axes3D

import tensorflow_hub as hub
import tensorflow.nn as nn

import torch 
import torch.nn 

from inspect import signature
import time
import argparse

import os
import datetime
import pandas as pd
import tensorflow as tf

from scipy.special import legendre, binom


#generating the dataset: high dimensional data from low dimensions, legendre polynomials from Lorenz encoder

def lorenz_dynamics(s0, t, sigma=10, beta=8/3, rho=28):
  "simulating the dynamics"
  "s0: list, 3 initial conditions"
  "t = list, time points"
  "output: arrays of trajectory, with first, 2nd derivatives"

  f = lambda s, t: [ -1*sigma*(s[0]-s[1]), rho*s[0]-s[1]-s[0]*s[2],
                    s[0]*s[1]-beta*s[2]]
  df = lambda s, ds, t: [sigma*(ds[1]-ds[0]),
                         rho*ds[0]-ds[1]-ds[0]*s[2]-s[0]*ds[2],
                         ds[0]*s[1]+s[0]*ds[1]-beta*ds[2]]

  s = odeint(f, s0, t)

  dt = t[1]-t[0]

  ds = np.array([f(s[i],dt) for i in range(t.size)])
  dds = np.array([df(s[i], ds[i], dt) for i in range(t.size)])

  return s, ds, dds


def generate_lorenz(initials, t, n, linear=True, normalization=None, sigma=10,
                    beta=8/3, rho=28):
  "generate the high dimensional dataset"
  "initials = kx3 array of k initial conditions"
  "t = timesteps"
  "n = size of high dim dataset"
  "linear = is dataset linear combination of lorenz? or include cubic modes"
  "output: big dictionary, x has dimension n"
  "x is high dim dataset, s is low"

  n_ics = initials.shape[0]
  nsteps = t.size
  dt = t[1]-t[0]

  d=3

  setup = np.array([lorenz_dynamics(a,t) for a in initials])
  s = setup[:, 0] #shape (n_ics, nsteps, d)
  ds = setup[:, 1]
  dds = setup[:, 2]
  

  if normalization is not None:
    s *= normalization
    ds *= normalization
    dds *= normalization

  L = 1
  y_spatial = np.linspace(-1*L, L, n)

  modes = np.array([legendre(i)(y_spatial) for i in range(2*d)]) #shape(2d, n)

#figuring stuff out

  s_lin = np.repeat(s, n).reshape(n_ics, nsteps, d, n)
  s_sq = np.array([a ** 2 for a in s_lin])
  s_cub = np.array([a ** 3 for a in s_lin])

  ds_lin = np.repeat(ds, n).reshape(n_ics, nsteps, d,n)
  ds_sq = np.array([a ** 2 for a in ds_lin])
  dds_lin = np.repeat(dds, n).reshape(n_ics, nsteps, d,n)

  x1 = np.multiply(s_lin[:,:,0, :], modes[0])
  x2 = np.multiply(s_lin[:,:,1, :], modes[1])
  x3 = np.multiply(s_lin[:,:,2, :], modes[2])
  x4 = np.multiply(s_cub[:,:,0, :], modes[3])
  x5 = np.multiply(s_cub[:,:,1, :], modes[4])
  x6 = np.multiply(s_cub[:,:,2, :], modes[5])

  dx1 = np.multiply(ds_lin[:,:,0, :], modes[0])
  dx2 = np.multiply(ds_lin[:,:,1, :], modes[1])
  dx3 = np.multiply(ds_lin[:,:,2, :], modes[2])
  dx4 = np.multiply(np.multiply(ds_lin[:,:,0, :], modes[3]), s_sq[:,:,0,:])
  dx5 = np.multiply(np.multiply(ds_lin[:,:,1, :], modes[4]), s_sq[:,:,2,:])
  dx6 = np.multiply(np.multiply(ds_lin[:,:,2, :], modes[5]), s_sq[:,:,1,:])

  ddx1 = np.multiply(dds_lin[:,:,0, :], modes[0])
  ddx2 = np.multiply(dds_lin[:,:,1, :], modes[1])
  ddx3 = np.multiply(dds_lin[:,:,2, :], modes[2])
  ddx4 = np.multiply(dds_lin[:,:,0, :], modes[3])
  ddx5 = np.multiply(dds_lin[:,:,1, :], modes[4])
  ddx6 = np.multiply(dds_lin[:,:,2, :], modes[5])


  x = x1 + x2 + x3
  dx = dx1 + dx2 + dx3
  ddx = ddx1 + ddx2 + ddx3

  if not linear:
    x += x4 + x5 + x6
    dx +=  3* (dx4+ + dx5 + dx6)   
    ddx += np.multiply(ddx4,s_sq[:,:,0,:]) \
    +2*np.multiply(ds_sq[:,:,0,:], s_lin[:,:,0,:]) \
        +np.multiply(ddx5,s_sq[:,:,1,:]) \
    +2*np.multiply(ds_sq[:,:,1,:], s_lin[:,:,1,:]) \
        +np.multiply(ddx6,s_sq[:,:,2,:]) \
    +2*np.multiply(ds_sq[:,:,2,:], s_lin[:,:,2,:]) 
  

  data = {}
  data["t"] = t
  data["y_spatial"] = y_spatial
  data["modes"] = modes
  data["x"] = x
  data["dx"] = dx
  data["ddx"] = ddx
  data["s"] = s
  data["ds"] = ds
  data["dds"] = dds

  return data


def generate_dataset(n_ics, noise_strength=0):
  t = np.arange(0, 10, 0.01) #returns array, no len, use size instead
  steps = t.size
  input_dim = 128

    
  ic_means = np.array([0,0,25])
  ic_widths = 2*np.array([18,24,36])

  # training data
  ics = ic_widths*(np.random.rand(n_ics, 3)-.5) + ic_means
  data = generate_lorenz(ics, t, input_dim, linear=False, normalization=np.array([1/40,1/40,1/40]))

  data['x'] = data['x'].reshape((-1,input_dim)) + noise_strength*np.random.randn(steps*n_ics,input_dim)
  data['dx'] = data['dx'].reshape((-1,input_dim)) + noise_strength*np.random.randn(steps*n_ics,input_dim)
  data['ddx'] = data['ddx'].reshape((-1,input_dim)) + noise_strength*np.random.randn(steps*n_ics,input_dim)


  return data

In [4]:
#generating data 

noise_strength= 1e-6
training_data = generate_dataset(128, noise_strength=noise_strength)
validation_data = generate_dataset(20, noise_strength=noise_strength)

params={}

params['input_dim'] = 128
params['latent_dim'] = 3
params['model_order'] = 1
params['poly_order'] = 3
params['include_sine'] = False



params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500

params['coefficient_initialization'] = 'constant'

params['loss_weight_decoder'] = 1
params['loss_weight_eql_z'] = 1e-3
params["loss_weight_eql_x"] = 1e-3
params["loss_weight_eql_regularization"] = 1e-3
params['activation'] = 'sigmoid'

params['widths'] = [64, 32]
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 32
params['learning_rate'] = 1e-4
params['data_path'] = os.getcwd() + '/'

params['print_progress'] = True
params['print_frequency'] = 100
params['max_epochs'] = 1001
params['refinement_epochs'] = 1001

#eql things

params["results_dir"] = "results/benchmark/test"
params["n_layers"] = 2
params["reg_weight"] = 1e-2
params["learning_rate_1"] = 1e-2
params["n_epochs1"] = 20001
params["n_epochs2"] = 10001
params["split"] = 0.8

# EQL things

functions

In [ ]:
import sympy as sp

class BaseFunction:
    """Abstract class for primitive functions"""
    def __init__(self, norm=1):
        self.norm = norm

    def sp(self, x):
        """Sympy implementation"""
        return None

    def tf(self, x):
        """Automatically convert sympy to TensorFlow"""
        z = sp.symbols('z')
        return sp.utilities.lambdify(z, self.sp(z), 'tensorflow')(x)

    def np(self, x):
        """Automatically convert sympy to numpy"""
        z = sp.symbols('z')
        return sp.utilities.lambdify(z, self.sp(z), 'numpy')(x)

    def name(self, x):
        return str(self.sp)


class Constant(BaseFunction):
    def tf(self, x):
        return tf.ones_like(x)

    def sp(self, x):
        return 1

    def np(self, x):
        return np.ones_like


class Identity(BaseFunction):
    def tf(self, x):
        return tf.identity(x) / self.norm

    def sp(self, x):
        return x / self.norm

    def np(self, x):
        return np.array(x) / self.norm


class Square(BaseFunction):
    def tf(self, x):
        return tf.square(x) / self.norm

    def sp(self, x):
        return x ** 2 / self.norm

    def np(self, x):
        return np.square(x) / self.norm


class Pow(BaseFunction):
    def __init__(self, power, norm=1):
        BaseFunction.__init__(self, norm=norm)
        self.power = power

    def sp(self, x):
        return x ** self.power / self.norm

    def tf(self, x):
        return tf.pow(x, self.power) / self.norm


class Sin(BaseFunction):
    def sp(self, x):
        return sp.sin(x * 2*2*np.pi) / self.norm


class Sigmoid(BaseFunction):
    # def tf(self, x):
    #     return tf.sigmoid(x) / self.norm

    def sp(self, x):
        return 1 / (1 + sp.exp(-20*x)) / self.norm

    def np(self, x):
        return 1 / (1 + np.exp(-20*x)) / self.norm

    def name(self, x):
        return "sigmoid(x)"


class Exp(BaseFunction):
    def __init__(self, norm=np.e):
        super().__init__(norm)

    def sp(self, x):
        return (sp.exp(x) - 1) / self.norm


class Log(BaseFunction):
    def sp(self, x):
        return sp.log(sp.Abs(x)) / self.norm


class BaseFunction2:
    """Abstract class for primitive functions with 2 inputs"""
    def __init__(self, norm=1.):
        self.norm = norm

    def sp(self, x, y):
        """Sympy implementation"""
        return None

    def tf(self, x, y):
        """Automatically convert sympy to TensorFlow"""
        a, b = sp.symbols('a b')
        return sp.utilities.lambdify([a, b], self.sp(a, b), 'tensorflow')(x, y)

    def np(self, x, y):
        """Automatically convert sympy to numpy"""
        a, b = sp.symbols('a b')
        return sp.utilities.lambdify([a, b], self.sp(a, b), 'numpy')(x, y)

    def name(self, x, y):
        return str(self.sp)


class Product(BaseFunction2):
    def __init__(self, norm=0.1):
        super().__init__(norm=norm)

    def sp(self, x, y):
        return x*y / self.norm


def count_inputs(funcs):
    i = 0
    for func in funcs:
        if isinstance(func, BaseFunction):
            i += 1
        elif isinstance(func, BaseFunction2):
            i += 2
    return i


def count_double(funcs):
    i = 0
    for func in funcs:
        if isinstance(func, BaseFunction2):
            i += 1
    return i


# default_func = [
#     Constant(),
#     Constant(),
#     Identity(),
#     Identity(),
#     Square(),
#     Square(),
#     Sin(),
#     Sigmoid(),
# ]

default_func = [
    *[Constant()] * 2,
    *[Identity()] * 4,
    *[Square()] * 4,
    *[Sin()] * 2,
    *[Exp()] * 2,
    *[Sigmoid()] * 2,
    *[Product()] * 2,
]

In [ ]:
def apply_activation(W, funcs, n_double=0):
    """Given an (n, m) matrix W and (m) vector of funcs, apply funcs to W.

    Arguments:
        W:  (n, m) matrix
        funcs: list of activation functions (SymPy functions)
        n_double:   Number of activation functions that take in 2 inputs

    Returns:
        SymPy matrix with 1 column that represents the output of applying the activation functions.
    """
    W = sym.Matrix(W)
    if n_double == 0:
        for i in range(W.shape[0]):
            for j in range(W.shape[1]):
                W[i, j] = funcs[j](W[i, j])
    else:
        W_new = W.copy()
        out_size = len(funcs)
        for i in range(W.shape[0]):
            in_j = 0
            out_j = 0
            while out_j < out_size - n_double:
                W_new[i, out_j] = funcs[out_j](W[i, in_j])
                in_j += 1
                out_j += 1
            while out_j < out_size:
                W_new[i, out_j] = funcs[out_j](W[i, in_j], W[i, in_j+1])
                in_j += 2
                out_j += 1
        for i in range(n_double):
            W_new.col_del(-1)
        W = W_new
    return W


def sym_pp(W_list, funcs, var_names, threshold=0.01, n_double=0):
    """Pretty print the hidden layers (not the last layer) of the symbolic regression network

    Arguments:
        W_list: list of weight matrices for the hidden layers
        funcs:  list of lambda functions using sympy. has the same size as W_list[i][j, :]
        var_names: list of strings for names of variables
        threshold: threshold for filtering expression. set to 0 for no filtering.
        n_double:   Number of activation functions that take in 2 inputs

    Returns:
        Simplified sympy expression.
    """
    vars = []
    for var in var_names:
        if isinstance(var, str):
            vars.append(sym.Symbol(var))
        else:
            vars.append(var)
    expr = sym.Matrix(vars).T
    W_list = np.asarray(W_list)
    for W in W_list:
        W = filter_mat(sym.Matrix(W), threshold=threshold)
        expr = expr * W
        expr = apply_activation(expr, funcs, n_double=n_double)
    # expr = expr * W_list[-1]
    return expr


def last_pp(eq, W):
    """Pretty print the last layer."""
    return eq * filter_mat(sym.Matrix(W))


def network(weights, funcs, var_names, threshold=0.01):
    """Pretty print the entire symbolic regression network.

    Arguments:
        weights: list of weight matrices for the entire network
        funcs:  list of lambda functions using sympy. has the same size as W_list[i][j, :]
        var_names: list of strings for names of variables
        threshold: threshold for filtering expression. set to 0 for no filtering.

    Returns:
        Simplified sympy expression."""
    n_double = count_double(funcs)
    funcs = [func.sp for func in funcs]

    expr = sym_pp(weights[:-1], funcs, var_names, threshold=threshold, n_double=n_double)
    expr = last_pp(expr, weights[-1])
    expr = expr[0, 0]
    return expr


def filter_mat(mat, threshold=0.01):
    """Remove elements of a matrix below a threshold."""
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            if abs(mat[i, j]) < threshold:
                mat[i, j] = 0
    return mat


def filter_expr(expr, threshold=0.01):
    """Remove additive terms with coefficient below threshold
    TODO: Make more robust. This does not work in all cases."""
    expr_new = sym.Integer(0)
    for arg in expr.args:
        if arg.is_constant() and abs(arg) > threshold:   # hack way to check if it's a number
            expr_new = expr_new + arg
        elif not arg.is_constant() and abs(arg.args[0]) > threshold:
            expr_new = expr_new + arg
    return expr_new


def filter_expr2(expr, threshold=0.01):
    """Sets all constants under threshold to 0
    TODO: Test"""
    for a in sym.preorder_traversal(expr):
        if isinstance(a, sym.Float) and a < threshold:
            expr = expr.subs(a, 0)
    return expr


symbolic network

In [ ]:
# Constants for L0 Regularization
BETA = 2 / 3
GAMMA = -0.1
ZETA = 1.1
EPSILON = 1e-6


class SymbolicLayer:
    """Neural network layer for symbolic regression where activation functions correspond to primitive functions.
    Can take multi-input activation functions (like multiplication)"""
    def __init__(self, funcs=None, initial_weight=None, variable=False, init_stddev=0.1):
        """

        funcs: List of activation functions, using utils.functions
        initial_weight: (Optional) Initial value for weight matrix
        variable: Boolean of whether initial_weight is a variable or not
        init_stddev: (Optional) if initial_weight isn't passed in, this is standard deviation of initial weight
        """

        if funcs is None:
            funcs = default_func
        self.initial_weight = initial_weight
        self.W = None       # Weight matrix
        self.built = False  # Boolean whether weights have been initialized
        if self.initial_weight is not None:     # use the given initial weight
            with tf.name_scope("symbolic_layer"):
                if not variable:
                    self.W = tf.Variable(self.initial_weight)
                else:
                    self.W = self.initial_weight
            self.built = True

        self.output = None  # Tensorflow tensor for layer output
        self.init_stddev = init_stddev
        self.n_funcs = len(funcs)           # Number of activation functions (and number of layer outputs)
        self.funcs = [func.tf for func in funcs]        # Convert functions to list of Tensorflow functions
        self.n_double = count_double(funcs)   # Number of activation functions that take 2 inputs
        self.n_single = self.n_funcs - self.n_double    # Number of activation functions that take 1 input

        self.out_dim = self.n_funcs + self.n_double

    def build(self, in_dim):
        """Initialize weight matrix"""
        self.W = tf.compat.v1.Variable(tf.random_normal(shape=[in_dim, self.out_dim], stddev=self.init_stddev)) #different than v2
        print(self.W)
        self.built = True

    def __call__(self, x):
        """Multiply by weight matrix and apply activation units"""
        with tf.name_scope("symbolic_layer"):
            if not self.built:
                self.build(x.shape[1].value)    # First dimension is batch size
            g = tf.matmul(x, self.W)  # shape = (?, self.size)
            self.output = []

            in_i = 0    # input index
            out_i = 0   # output index
            # Apply functions with only a single input
            while out_i < self.n_single:
                self.output.append(self.funcs[out_i](g[:, in_i]))
                in_i += 1
                out_i += 1
            # Apply functions that take 2 inputs and produce 1 output
            while out_i < self.n_funcs:
                self.output.append(self.funcs[out_i](g[:, in_i], g[:, in_i+1]))
                in_i += 2
                out_i += 1
            self.output = tf.stack(self.output, axis=1)
            return self.output

    def get_weight(self):
        return self.W


class SymbolicLayerBias(SymbolicLayer):
    """SymbolicLayer with a bias term"""
    def __init__(self, funcs=None, initial_weight=None, variable=False, init_stddev=0.1):
        super().__init__(funcs, initial_weight, variable, init_stddev)
        self.b = None

    def build(self, in_dim):
        super().build(in_dim)
        self.b = tf.compat.v1.Variable(tf.ones(shape=self.n_funcs) * 0.01)

    def __call__(self, x):
        """Multiply by weight matrix and apply activation units"""
        super().__call__(x)
        self.output += self.b
        return self.output


class SymbolicNet:
    """Symbolic regression network with multiple layers. Produces one output."""
    def __init__(self, symbolic_depth, funcs=None, initial_weights=None, initial_bias=None,
                 variable=False, init_stddev=0.1):
        self.depth = symbolic_depth     # Number of hidden layers
        self.funcs = funcs
        self.shape = (None, 1)
        if initial_weights is not None:
            self.symbolic_layers = [SymbolicLayer(funcs=funcs, initial_weight=initial_weights[i], variable=variable)
                                    for i in range(self.depth)]
            if not variable:
                self.output_weight = tf.compat.v1.Variable(initial_weights[-1])
            else:
                self.output_weight = initial_weights[-1]
        else:
            # Each layer initializes its own weights
            if isinstance(init_stddev, list):
                self.symbolic_layers = [SymbolicLayer(funcs=funcs, init_stddev=init_stddev[i]) for i in range(self.depth)]
            else:
                self.symbolic_layers = [SymbolicLayer(funcs=funcs, init_stddev=init_stddev) for _ in range(self.depth)]
            # Initialize weights for last layer (without activation functions)
            self.output_weight = tf.compat.v1.Variable(tf.random_uniform(shape=(self.symbolic_layers[-1].n_funcs, 1)))

    def build(self, input_dim):
        in_dim = input_dim
        for i in range(self.depth):
            self.symbolic_layers[i].build(in_dim)
            in_dim = self.symbolic_layers[i].n_funcs

    def __call__(self, input):
        self.shape = (int(input.shape[1]), 1)     # Dimensionality of the input
        h = input
        # Building hidden layers
        for i in range(self.depth):
            h = self.symbolic_layers[i](h)
        # Final output (no activation units) of network
        h = tf.matmul(h, self.output_weight)
        return h

    def get_weights(self):
        """Return list of weight matrices"""
        # First part is iterating over hidden weights. Then append the output weight.
        return [self.symbolic_layers[i].W for i in range(self.depth)] + [self.output_weight]


class MaskedSymbolicNet(SymbolicNet):
    """Symbolic regression network where weights below a threshold are set to 0 and frozen. In other words, we apply
    a mask to the symbolic network to fine-tune the non-zero weights."""
    def __init__(self, sess, sr_unit, threshold=0.01):
        # weights = sr_unit.get_weights()
        # masked_weights = []
        # for w_i in weights:
        #     # w_i = tf.where(tf.abs(w_i) < threshold, tf.zeros_like(w_i), w_i)
        #     # w_i = tf.where(tf.abs(w_i) < threshold, tf.stop_gradient(w_i), w_i)
        #     masked_weights.append(w_i)

        weights = sr_unit.get_weights()
        masked_weights = []
        for w_i in weights:
            mask = tf.constant(sess.run(tf.math.abs(w_i) > threshold), dtype=tf.float32)
            masked_weights.append(tf.math.multiply(w_i, mask))

        super().__init__(sr_unit.depth, funcs=sr_unit.funcs, initial_weights=masked_weights, variable=True)
        self.sr_unit = sr_unit


class SymbolicLayerL0(SymbolicLayer):
    def __init__(self, funcs=None, initial_weight=None, variable=False, init_stddev=0.1,
                 bias=False, droprate_init=0.5, lamba=1.):
        super().__init__(funcs, initial_weight, variable, init_stddev) #super computes thefunction itself
        self.droprate_init = droprate_init if droprate_init != 0 else 0.5
        self.use_bias = bias
        self.lamba = lamba
        self.bias = None
        self.qz_log_alpha = None
        self.in_dim = None
        self.eps = None

    def build(self, in_dim):
        with tf.name_scope("symbolic_layer"):
            self.in_dim = in_dim
            if self.W is None:
                self.W = tf.Variable(tf.random.normal(shape=[in_dim, self.out_dim], stddev=self.init_stddev))
            if self.use_bias:
                self.bias = tf.Variable(0.1*tf.ones((1, self.out_dim)))
            self.qz_log_alpha = tf.Variable(tf.random.normal(shape=(in_dim, self.out_dim),
                                                             mean=tf.math.log(1-self.droprate_init) - tf.math.log(self.droprate_init),
                                                             stddev=1e-2))

    def quantile_concrete(self, u):
        """Quantile, aka inverse CDF, of the 'stretched' concrete distribution"""
        y = tf.sigmoid((tf.math.log(u) - tf.math.log(1.0-u) + self.qz_log_alpha) / BETA)
        return y * (ZETA - GAMMA) + GAMMA

    def sample_u(self, shape, reuse_u=False):
        """Uniform random numbers for concrete distribution"""
        # print("Hello")
        if self.eps is None or not reuse_u:
            self.eps = tf.random.uniform(shape=shape, minval=EPSILON, maxval=1.0 - EPSILON)
        return self.eps

    def sample_z(self, batch_size, sample=True):
        """Use the hard concrete distribution as described in https://arxiv.org/abs/1712.01312"""
        if sample:
            eps = self.sample_u((batch_size, self.in_dim, self.out_dim))
            z = self.quantile_concrete(eps)
            return tf.clip_by_value(z, 0, 1)
        else:   # Mean of the hard concrete distribution
            pi = tf.sigmoid(self.qz_log_alpha)
            return tf.clip_by_value(pi * (ZETA - GAMMA) + GAMMA, clip_value_min=0.0, clip_value_max=1.0)

    def get_z_mean(self):
        """Mean of the hard concrete distribution"""
        pi = tf.sigmoid(self.qz_log_alpha)
        return tf.clip_by_value(pi * (ZETA - GAMMA) + GAMMA, clip_value_min=0.0, clip_value_max=1.0)

    def sample_weights(self, reuse_u=False):
        z = self.quantile_concrete(self.sample_u((self.in_dim, self.out_dim), reuse_u=reuse_u))
        mask = tf.clip_by_value(z, clip_value_min=0.0, clip_value_max=1.0)
        return mask * self.W

    def get_weight(self):
        """Deterministic value of weight based on mean of z"""
        return self.W * self.get_z_mean()

    def loss(self):
        """Regularization loss term"""
        return tf.reduce_sum(tf.sigmoid(self.qz_log_alpha - BETA * tf.math.log(-GAMMA / ZETA)))

    def __call__(self, x, sample=True, reuse_u=False):
        """Multiply by weight matrix and apply activation units"""

        with tf.name_scope("symbolic_layer"):
            if self.W is None or self.qz_log_alpha is None:
                self.build(x.shape[1])

            if sample:
                h = tf.matmul(x, self.sample_weights(reuse_u=reuse_u))
            else:
                print("correct")
                w = self.get_weight()
                h = tf.matmul(x, w)

            if self.use_bias:
                h = h + self.bias

            # shape of h = (?, self.n_funcs)

            self.output = []
            # apply a different activation unit to each column of h
            in_i = 0    # input index
            out_i = 0   # output index
            # Apply functions with only a single input
            while out_i < self.n_single:
                self.output.append(self.funcs[out_i](h[:, in_i]))
                in_i += 1
                out_i += 1
            # Apply functions that take 2 inputs and produce 1 output
            while out_i < self.n_funcs:
                self.output.append(self.funcs[out_i](h[:, in_i], h[:, in_i+1]))
                in_i += 2
                out_i += 1
            self.output = tf.stack(self.output, axis=1)
            return self.output


class SymbolicNetL0(SymbolicNet):
    """Symbolic regression network with multiple layers. Produces one output."""
    def __init__(self, symbolic_depth, funcs=None, initial_weights=None, initial_bias=None,
                 variable=False, init_stddev=0.1):
        super().__init__(symbolic_depth, funcs, initial_weights, initial_bias, variable, init_stddev)
        if initial_weights is not None:
            self.symbolic_layers = [SymbolicLayerL0(funcs=funcs, initial_weight=initial_weights[i], variable=variable)
                                    for i in range(self.depth)]
            if not variable:
                self.output_weight = tf.Variable(initial_weights[-1])
            else:
                self.output_weight = initial_weights[-1]
        else:
            # Each layer initializes its own weights
            if isinstance(init_stddev, list):
                self.symbolic_layers = [SymbolicLayerL0(funcs=funcs, init_stddev=init_stddev[i])
                                        for i in range(self.depth)]
            else:
                self.symbolic_layers = [SymbolicLayerL0(funcs=funcs, init_stddev=init_stddev)
                                        for _ in range(self.depth)]
            # Initialize weights for last layer (without activation functions)
            self.output_weight = tf.Variable(tf.random_uniform(shape=(self.symbolic_layers[-1].n_funcs, 1)))

    def __call__(self, input, sample=True, reuse_u=False):
        tf.compat.v1.enable_eager_execution()
        self.shape = (int(input.shape[1]), 1)     # Dimensionality of the input
        # connect output from previous layer to input of next layer
        h = input
        saved_h = [h]
        for i in range(self.depth):
            if i==0 or i == self.depth-1:
                h = self.symbolic_layers[i](h, sample=sample, reuse_u=reuse_u)
            else:
                h = self.symbolic_layers[i](h, sample=sample, reuse_u=reuse_u)
                h = tf.concat([h, saved_h[-1]], 0)
            
            saved_h.append(h)

        # Final output (no activation units) of network
        h = tf.matmul(h, self.output_weight)

        return h


    def get_loss(self):
        return tf.reduce_sum([self.symbolic_layers[i].loss() for i in range(self.depth)])

    def get_weights(self):
        return self.get_symbolic_weights() + [self.get_output_weight()]

    def get_symbolic_weights(self):
        return [self.symbolic_layers[i].get_weight() for i in range(self.depth)]

    def get_output_weight(self):
        return self.output_weight


class SymbolicCell(tf.keras.layers.SimpleRNNCell):
    """cell for use with tf.keras.layers.RNN, allowing us to build a recurrent network with the EQL network.
    This is used for the propagating decoder in the dynamics architecture.
    Assume two state variables: position and velocity."""
    state_size = 2
    output_size = 2

    def __init__(self, sym1, sym2):
        # units: dimensionality of output space
        super().__init__(units=self.output_size)
        self.sym1 = sym1
        self.sym2 = sym2

    def call(self, inputs, state, training=None):
        """
        Arguments:
            inputs (at time t): shape (batch, feature)
            state: [x], shape(x)=(batch, feature)
            training:   Ignore this
        """
        full_input = state[0] + inputs[:, :2]
        full_input = tf.concat([full_input, inputs[:, 2:4]], axis=1)
        output = tf.concat([self.sym1(full_input), self.sym2(full_input)], axis=1)
        next_state = output
        return output, next_state

benchmark utils

In [ ]:
var_names = ["x", "y", "z"]

def l12_smooth(input_tensor, a=0.05):
    """Smoothed L1/2 norm"""
    if type(input_tensor) == list:
        return sum([l12_smooth(tensor) for tensor in input_tensor])

    smooth_abs = tf.where(tf.abs(input_tensor) < a,
                          tf.pow(input_tensor, 4)/(-8*a**3) + tf.square(input_tensor)*3/4/a + 3*a/8,
                          tf.abs(input_tensor))
    return tf.reduce_sum(tf.sqrt(smooth_abs))

class Benchmark:
    """Benchmark object just holds the results directory (results_dir) to save to and the hyper-parameters. So it is
    assumed all the results in results_dir share the same hyper-parameters. This is useful for benchmarking multiple
    functions with the same hyper-parameters."""
    def __init__(self, results_dir, n_layers=2, reg_weight=5e-3, learning_rate=1e-2,
                 n_epochs1=10001, n_epochs2=10001):
        """Set hyper-parameters"""
        self.activation_funcs = [
            *[functions.Constant()] * 2,
            *[functions.Identity()] * 4,
            *[functions.Square()] * 4,
            *[functions.Sin()] * 2,
            *[functions.Exp()] * 2,
            *[functions.Sigmoid()] * 2,
            *[functions.Product()] * 2
        ]

        self.n_layers = n_layers              # Number of hidden layers
        self.reg_weight = reg_weight     # Regularization weight
        self.learning_rate = learning_rate
        self.summary_step = 1000    # Number of iterations at which to print to screen
        self.n_epochs1 = n_epochs1
        self.n_epochs2 = n_epochs2

        if not os.path.exists(results_dir):
            os.makedirs(results_dir)
        self.results_dir = results_dir

        # Save hyperparameters to file
        result = {
            "learning_rate": self.learning_rate,
            "summary_step": self.summary_step,
            "n_epochs1": self.n_epochs1,
            "n_epochs2": self.n_epochs2,
            "activation_funcs_name": [func.name for func in self.activation_funcs],
            "n_layers": self.n_layers,
            "reg_weight": self.reg_weight,
        }
        with open(os.path.join(self.results_dir, 'params.pickle'), "wb+") as f:
            pickle.dump(result, f)

    def benchmark(self, func_dim, func_name, trials, x_comp, y_comp, split): #x_comp, y_comp are compilations of xset, yset
        """Benchmark the EQL network on data generated by the given function. Print the results ordered by test error.

        Arguments:
            func_dim: dimension of dataset
            func_name: string that describes the function - this will be the directory name
            trials: number of trials to train from scratch. Will save the results for each trial.
        """

        print("Starting benchmark for function:\t%s" % func_name)
        print("==============================================")

        # Create a new sub-directory just for the specific function
        func_dir = os.path.join(self.results_dir, func_name)
        if not os.path.exists(func_dir):
            os.makedirs(func_dir)

        ##################### Train network! #######################
        expr_list, error_test_list = self.train(x_comp=x_comp, y_comp=y_comp, split=split,
                                                func_dim=func_dim, func_name=func_name,
                                                trials=trials, func_dir=func_dir)
        
        #instead of being lists, let's just do them normally

        # Sort the results by test error (increasing) and print them to file
        # This allows us to easily count how many times it fit correctly.


        error_expr_sorted = sorted(zip(error_test_list, expr_list))     # List of (error, expr)
        error_test_sorted = [x for x, _ in error_expr_sorted]   # Separating out the errors
        expr_list_sorted = [x for _, x in error_expr_sorted]    # Separating out the expr

        fi = open(os.path.join(self.results_dir, 'eq_summary.txt'), 'a')
        fi.write("\n{}\n".format(func_name))
        for i in range(trials):
            fi.write("[%f]\t\t%s\n" % (error_test_sorted[i], str(expr_list_sorted[i])))
        fi.close()

        #knowing that the compiled x gives the real x... 

        final_expression = expr_list_sorted[0] #sympy
        final_predict = generate_results(final_expression, func_dim, x_comp)

        return final_predict
              
    def generate_results(self, expression, func_dim, x_comp):
        # full_x = np.append(x_comp[0], x_comp[1], axis=0)

        returnList = np.array([[]])
        n = x_comp.get_shape().as_list()[0]

        for i in range(n):
          a = x_comp[i][0]
          b = x_comp[i][1]
          c = x_comp[i][2]

          evaluate = np.array([[final_expression.sp.subs([(x,a),(y,b),(z,c)])]])
          returnList = np.append(returnList, evaluate, axis=0)

        return returnList

    def train(self, x_comp, y_comp, split, func_dim, func_name='', trials=1, func_dir='results/test'): # trains itself, this is the true L0 one
        """Train the network to find a given function"""

        n = x_comp.get_shape().as_list()[0]

        if n == None:
          split_index = None
        else:
          split_index = int(split * n)

        x_dim = func_dim #len(signature(func).parameters)  # Number of input arguments to the function
        # Generate training data and test data

        x, x_test = x_comp[:split_index], x_comp[split_index:]
        y, y_test = y_comp[:split_index], y_comp[split_index:]

        # x, y = generate_data(func, N_TRAIN)
        # x_val, y_val = generate_data(func, N_VAL)
        # x_test, y_test = generate_data(func, N_TEST, range_min=DOMAIN_TEST[0], range_max=DOMAIN_TEST[1])

        # Setting up the symbolic regression network
        x_placeholder = tf.compat.v1.placeholder(shape=(None, x_dim), dtype=tf.float32)
        width = len(self.activation_funcs)
        n_double = count_double(self.activation_funcs)
        sym = SymbolicNetL0(self.n_layers, funcs=self.activation_funcs,
                            initial_weights=[
                                                 tf.random.truncated_normal([x_dim, width + n_double], stddev=init_sd_first, dtype=tf.float32), #removed the dtypes
                                                 tf.random.truncated_normal([width, width + n_double], stddev=init_sd_middle, dtype=tf.float32),
                                                 tf.random.truncated_normal([width, width + n_double], stddev=init_sd_middle, dtype=tf.float32),
                                                 tf.random.truncated_normal([width, 1], stddev=init_sd_last, dtype=tf.float32)
                                             ], )
        y_hat = sym(x_placeholder)

        # Label and errors
        error = tf.keras.losses.mean_squared_error(y_true=y, y_pred=y_hat)
        error_test = tf.keras.losses.mean_squared_error(y_true=y_test, y_pred=y_hat)
        # Regularization oscillates as a function of epoch.
        reg_loss = sym.get_loss()
        loss = error + self.reg_weight * reg_loss

        # Training
        learning_rate = tf.compat.v1.placeholder(tf.float32)
        opt = tf.compat.v1.train.RMSPropOptimizer(learning_rate=learning_rate)
        train = opt.minimize(loss)

        loss_list = []  # Total loss (MSE + regularization)
        error_list = []     # MSE
        reg_list = []       # Regularization
        error_test_list = []    # Test error

        error_test_final = []
        eq_list = []

        # Only take GPU memory as needed - allows multiple jobs on a single GPU
        config = tf.compat.v1.ConfigProto()
        config.gpu_options.allow_growth = True
        with tf.compat.v1.Session(config=config) as sess:
            for trial in range(trials):
                print("Training on function " + func_name + " Trial " + str(trial+1) + " out of " + str(trials))

                loss_val = np.nan
                # Restart training if loss goes to NaN (which happens when gradients blow up)
                while np.all(np.isnan(loss_val)):
                    sess.run(tf.compat.v1.global_variables_initializer())
                    # 1st stage of training with oscillating regularization weight
                    for i in range(self.n_epochs1):
                        feed_dict = {x_placeholder: x, learning_rate: self.learning_rate}
                        _ = sess.run(train, feed_dict=feed_dict)
                        if i % self.summary_step == 0:
                            loss_val, error_val, reg_val, = sess.run((loss, error, reg_loss), feed_dict=feed_dict)
                            error_test_val = sess.run(error_test, feed_dict={x_placeholder: x_test})

                            loss_val_avg = sum(loss_val) / len(loss_val)
                            error_test_val_avg = sum(error_test_val) / len(error_test_val)
                            error_val_avg = sum(error_val) / len(error_val)

                            print(error_val_avg)

                            print("Epoch: %d\tTotal training loss: %f\tTest error: %f" % (i, loss_val_avg, error_test_val_avg))
                            loss_list.append(loss_val_avg)
                            error_list.append(error_val_avg)
                            reg_list.append(reg_val)
                            error_test_list.append(error_test_val_avg)
                            if np.any(np.isnan(loss_val)):  # If loss goes to NaN, restart training
                                break

                # Print the expressions
                weights = sess.run(sym.get_weights())
                expr = network(weights, self.activation_funcs, var_names[:x_dim])
                print(expr)

                # Save results
                trial_file = os.path.join(func_dir, 'trial%d.pickle' % trial)

                results = {
                    "weights": weights,
                    "loss_list": loss_list,
                    "error_list": error_list,
                    "reg_list": reg_list,
                    "error_test": error_test_list,
                    "expr": expr
                }

                with open(trial_file, "wb+") as f:
                    pickle.dump(results, f)

                error_test_final.append(error_test_list[-1])
                eq_list.append(expr)

        return eq_list, error_test_final

BENCHMARK

# Network Below

In [ ]:
import pickle

tf.compat.v1.disable_eager_execution()

# N_TRAIN = 256       # Size of training dataset
# N_VAL = 100         # Size of validation dataset
# DOMAIN = (-1, 1)    # Domain of dataset
# # DOMAIN = np.array([[0, -1, -1], [1, 1, 1]])   # Use this format if each input variable has a different domain
# N_TEST = 100        # Size of test dataset
# DOMAIN_TEST = (-2, 2)   # Domain of test dataset - should be larger than training domain to test extrapolation
# NOISE_SD = 0        # Standard deviation of noise for training dataset
var_names = ["x", "y", "z"]

# Standard deviation of random distribution for weight initializations.
init_sd_first = 0.5
init_sd_last = 0.5
init_sd_middle = 0.5


# generate_data = benchmark.generate_data


class BenchmarkReal(Benchmark): # call BenchmarkReal.benchmark() to actually start things
    """Benchmark object just holds the results directory (results_dir) to save to and the hyper-parameters. So it is
    assumed all the results in results_dir share the same hyper-parameters. This is useful for benchmarking multiple
    functions with the same hyper-parameters."""
    def __init__(self, results_dir, n_layers=2, reg_weight=1e-2, learning_rate=1e-2,
                 n_epochs1=20001, n_epochs2=10001):
        """Set hyper-parameters"""
        self.activation_funcs = [
            *[Constant()] * 2,
            *[Identity()] * 4,
            *[Square()] * 4,
            *[Sin()] * 2,
            *[Exp()] * 2, #something wrong with the exponent
            *[Sigmoid()] * 2,
            *[Product()] * 2
        ]

        self.n_layers = n_layers              # Number of hidden layers
        self.reg_weight = reg_weight     # Regularization weight
        self.learning_rate = learning_rate
        self.summary_step = 1000    # Number of iterations at which to print to screen
        self.n_epochs1 = n_epochs1
        self.n_epochs2 = n_epochs2

        if not os.path.exists(results_dir):
            os.makedirs(results_dir)
        self.results_dir = results_dir

        # Save hyperparameters to file
        result = {
            "learning_rate": self.learning_rate,
            "summary_step": self.summary_step,
            "n_epochs1": self.n_epochs1,
            "n_epochs2": self.n_epochs2,
            "activation_funcs_name": [func.name for func in self.activation_funcs],
            "n_layers": self.n_layers,
            "reg_weight": self.reg_weight,
        }
        with open(os.path.join(self.results_dir, 'params.pickle'), "wb+") as f:
            pickle.dump(result, f)


eql_setup = BenchmarkReal(results_dir=params["results_dir"],
                          n_layers=params["n_layers"],
                          reg_weight=params["reg_weight"],
                          learning_rate=params["learning_rate_1"],
                          n_epochs1=params["n_epochs1"],
                          n_epochs2=params["n_epochs2"])


# going into the network is x, y, z autoencoder predict, coming out is equations
# predicting dx dy dz 

In [ ]:
#constructing network, replacing sindy with CNN
#fix all the np array tensor switcharound

#full network

# def y_composite(y, func_dim, splice):
#   n = splice*len(y)
#   returnList = []
#   for i in range(func_dim):
#     stored = []
#     arranged = y[:,[i]]
#     spliced_f = arranged[:n]
#     spliced_b = arranged[n:]

#     stored = [spliced_f, spliced_b]
#     returnList.append(stored)

#   return returnList
    
def full_network(params):
    """
    Define the full network architecture.

    Arguments:
        params - Dictionary object containing the parameters that specify the training.
        See README file for a description of the parameters.

    Returns:
        network - Dictionary containing the tensorflow objects that make up the network.
    """
    eql_test_frac = params["split"]

    input_dim = params['input_dim']
    latent_dim = params['latent_dim']
    activation = params['activation']
    poly_order = params['poly_order']
    if 'include_sine' in params.keys():
        include_sine = params['include_sine']
    else:
        include_sine = False
    model_order = params['model_order']

    network = {}

    x = tf.compat.v1.placeholder(tf.float32, shape=[None, input_dim], name='x')
    dx = tf.compat.v1.placeholder(tf.float32, shape=[None, input_dim], name='dx')

    z, x_decode, encoder_weights, encoder_biases, decoder_weights, decoder_biases = nonlinear_autoencoder(x, input_dim, latent_dim, params['widths'], activation=activation)

    
    #splicing and presetting

    # print(z)
    # z_shape = tf.compat.v1.placeholder(tf.float21, shape=[2])
    # sliced_z = z[:z_shape[0]]
    # length_splice_z = int(np.floor(z.shape[0] * eql_test_frac))
    # print(length_splice_z)

    # training_z = z[0:length_splice_z]
    # testing_z = z[length_splice_z:]

    dz = z_derivative(x, dx, encoder_weights, encoder_biases, activation=activation) # this just finds the z derivative assuming the same node transformation 


    y_comp_1, y_comp_2, y_comp_3 = dz[:,0], dz[:,1], dz[:,2]

    print(z)

    dz_predict_x = eql_setup.benchmark(func_dim=3, func_name="Lorenz_x", trials=1, x_comp=z, y_comp=y_comp_1, split=eql_test_frac)
    dz_predict_y = eql_setup.benchmark(func_dim=3, func_name="Lorenz_y", trials=1, x_comp=z, y_comp=y_comp_2, split=eql_test_frac)
    dz_predict_z = eql_setup.benchmark(func_dim=3, func_name="Lorenz_z", trials=1, x_comp=z, y_comp=y_comp_3, split=eql_test_frac)

    eql_predict = np.append(dz_predict_x, dz_predict_y, dz_predict_z, axis=1)

    dx_decode = z_derivative(z, eql_predict, decoder_weights, decoder_biases, activation=activation) #finds the decoded dx from the other direction

    network['x'] = x
    network['dx'] = dx
    network['z'] = z
    network['dz'] = dz
    network['x_decode'] = x_decode
    network['dx_decode'] = dx_decode
    network['encoder_weights'] = encoder_weights
    network['encoder_biases'] = encoder_biases
    network['decoder_weights'] = decoder_weights
    network['decoder_biases'] = decoder_biases
    network["dz_predict"] = eql_predict

    return network


#loss functions
def define_loss(network, params):
    """
    Create the loss functions.

    Arguments:
        network - Dictionary object containing the elements of the network architecture.
        This will be the output of the full_network() function.
    """
    x = network['x']
    x_decode = network['x_decode']
    dz = network['dz']
    dz_predict = network['dz_predict']
    dx = network['dx']
    dx_decode = network['dx_decode']

    losses = {}
    losses['decoder'] = tf.reduce_mean((x - x_decode)**2)
    losses['eql_z'] = tf.reduce_mean((dz - dz_predict)**2)
    losses['eql_x'] = tf.reduce_mean((dx - dx_decode)**2)
    loss = params['loss_weight_decoder'] * losses['decoder'] \
           + params['loss_weight_eql_z'] * losses['eql_z'] \
           + params['loss_weight_eql_x'] * losses['eql_x']

    loss_refinement = params['loss_weight_decoder'] * losses['decoder'] \
                      + params['loss_weight_eql_z'] * losses['eql_z'] \
                      + params['loss_weight_eql_x'] * losses['eql_x']

    return loss, losses, loss_refinement
    #loss_refinement is w/o sindy reg



#actual autoencoder parts
def nonlinear_autoencoder(x, input_dim, latent_dim, widths, activation='elu'):
  "returns a list of encoder, decoder weights and biases"
  if activation == 'relu':
    activation_function = tf.nn.relu
  elif activation == 'elu':
      activation_function = tf.nn.elu
  elif activation == 'sigmoid':
      activation_function = tf.sigmoid
  else:
    raise ValueError('invalid activation function')
    
  z,encoder_weights,encoder_biases = build_network_layers(x, input_dim, latent_dim, widths, activation_function, 'encoder')
  x_decode,decoder_weights,decoder_biases = build_network_layers(z, latent_dim, input_dim, widths[::-1], activation_function, 'decoder')


  return z, x_decode, encoder_weights, encoder_biases, decoder_weights, decoder_biases


def build_network_layers(input, input_dim, output_dim, widths, activation, name):
  "128 to 3 layers, with hidden layer of 376 inside"
  "W begins as a 128 x 128 shape, then gets multiplied to a 128, then etc etc"
  "widths = represent how many units are in each hidden layer"
  "builds an encoder, decoder"

  weights = []
  biases = []
  last_width=input_dim
  for i,n_units in enumerate(widths):
    W0 = tf.compat.v1.get_variable(name+'_W'+str(i), shape=[last_width,n_units],
            initializer=tf.random_normal_initializer(seed=1))
    
    W = tf.nn.dropout(W0, rate=0.3)
    b = tf.compat.v1.get_variable(name+'_b'+str(i), shape=[n_units],
            initializer=tf.constant_initializer(0.0))
    
    input = tf.matmul(input, W) + b
    if activation is not None:
      input = activation(input)

    last_width = n_units
    weights.append(W)
    biases.append(b)

  W0 = tf.compat.v1.get_variable(name+'_W'+str(len(widths)), shape=[last_width,output_dim],
      initializer=tf.random_normal_initializer(seed=1))
  b = tf.compat.v1.get_variable(name+'_b'+str(len(widths)), shape=[output_dim],
      initializer=tf.random_normal_initializer(seed=1))
  
  W = tf.nn.dropout(W0, rate=0.3)

  input = tf.matmul(input,W) + b
  weights.append(W)
  biases.append(b)
  return input, weights, biases


#z derivative
def z_derivative(input, dx, weights, biases, activation='elu'):
    dz = dx
    if activation == 'elu':
        for i in range(len(weights)-1):
            input = tf.matmul(input, weights[i]) + biases[i]
            dz = tf.multiply(tf.minimum(tf.exp(input),1.0),
                                  tf.matmul(dz, weights[i]))
            input = tf.nn.elu(input)
        dz = tf.matmul(dz, weights[-1])
    elif activation == 'relu':
        for i in range(len(weights)-1):
            input = tf.matmul(input, weights[i]) + biases[i]
            dz = tf.multiply(tf.to_float(input>0), tf.matmul(dz, weights[i]))
            input = tf.nn.relu(input)
        dz = tf.matmul(dz, weights[-1])
    elif activation == 'sigmoid':
        for i in range(len(weights)-1):
            input = tf.matmul(input, weights[i]) + biases[i]
            input = tf.sigmoid(input)
            dz = tf.multiply(tf.multiply(input, 1-input), tf.matmul(dz, weights[i]))
        dz = tf.matmul(dz, weights[-1])
    else:
        for i in range(len(weights)-1):
            dz = tf.matmul(dz, weights[i])
        dz = tf.matmul(dz, weights[-1])
    return dz


#eql things




In [ ]:
tf.compat.v1.disable_eager_execution()

def train_network(training_data, val_data, params):
    # SET UP NETWORK
    autoencoder_network = full_network(params)
    loss, losses, loss_refinement = define_loss(autoencoder_network, params)
    learning_rate = tf.compat.v1.placeholder(tf.float32, name='learning_rate')
    train_op = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
    train_op_refinement = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_refinement)
    saver = tf.compat.v1.train.Saver(var_list=tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.GLOBAL_VARIABLES))

    validation_dict = create_feed_dictionary(val_data, params, idxs=None)

    x_norm = np.mean(val_data['x']**2)
    nn_norm_x = np.mean(val_data['dx']**2)

    validation_losses = []

    print('TRAINING')
    with tf.compat.v1.Session() as sess:
        sess.run(tf.compat.v1.global_variables_initializer())
        for i in range(params['max_epochs']):
            for j in range(params['epoch_size']//params['batch_size']):
                batch_idxs = np.arange(j*params['batch_size'], (j+1)*params['batch_size'])
                train_dict = create_feed_dictionary(training_data, params, idxs=batch_idxs)
                sess.run(train_op, feed_dict=train_dict)
            
            if params['print_progress'] and (i % params['print_frequency'] == 0):
                validation_losses.append(print_progress(sess, i, loss, losses, train_dict, validation_dict, x_norm, nn_norm_x))

        print('REFINEMENT')
        for i_refinement in range(params['refinement_epochs']):
            for j in range(params['epoch_size']//params['batch_size']):
                batch_idxs = np.arange(j*params['batch_size'], (j+1)*params['batch_size'])
                train_dict = create_feed_dictionary(training_data, params, idxs=batch_idxs)
                sess.run(train_op_refinement, feed_dict=train_dict)
            
            if params['print_progress'] and (i_refinement % params['print_frequency'] == 0):
                validation_losses.append(print_progress(sess, i_refinement, loss_refinement, losses, train_dict, validation_dict, x_norm, nn_norm_x))

        saver.save(sess, params['data_path'] + params['save_name'])
        pickle.dump(params, open(params['data_path'] + params['save_name'] + '_params.pkl', 'wb'))
        final_losses = sess.run((losses['decoder'], losses['eql_x'], losses['eql_z']),
                                feed_dict=validation_dict)
        if params['model_order'] == 1:
            eql_norm_z = np.mean(sess.run(autoencoder_network['dz'], feed_dict=validation_dict)**2)
        else:
            eql_norm_z = np.mean(sess.run(autoencoder_network['ddz'], feed_dict=validation_dict)**2)

        results_dict = {}
        results_dict['num_epochs'] = i
        results_dict['x_norm'] = x_norm
        results_dict['eql_norm_x'] = eql_norm_x
        results_dict['eql_norm_z'] = eql_norm_z
        results_dict['loss_decoder'] = final_losses[0]
        results_dict['loss_decoder_eql'] = final_losses[1]
        results_dict['loss_eql'] = final_losses[2]
        results_dict['validation_losses'] = np.array(validation_losses)
        results_dict["autoencoder"] = autoencoder_network

        return results_dict


def print_progress(sess, i, loss, losses, train_dict, validation_dict, x_norm, nn_norm):
    """
    Print loss function values to keep track of the training progress.

    Arguments:
        sess - the tensorflow session
        i - the training iteration
        loss - tensorflow object representing the total loss function used in training
        losses - tuple of the individual losses that make up the total loss
        train_dict - feed dictionary of training data
        validation_dict - feed dictionary of validation data
        x_norm - float, the mean square value of the input
        nn_norm - float, the mean square value of the time derivatives of the input.
        Can be first or second order time derivatives depending on the model order.

    Returns:
        Tuple of losses calculated on the validation set.
    """
    training_loss_vals = sess.run((loss,) + tuple(losses.values()), feed_dict=train_dict)
    validation_loss_vals = sess.run((loss,) + tuple(losses.values()), feed_dict=validation_dict)
    print("Epoch %d" % i)
    print("   training loss {0}, {1}".format(training_loss_vals[0],
                                             training_loss_vals[1:]))
    print("   validation loss {0}, {1}".format(validation_loss_vals[0],
                                               validation_loss_vals[1:]))
    decoder_losses = sess.run((losses['decoder'], losses['eql_x']), feed_dict=validation_dict)
    loss_ratios = (decoder_losses[0]/x_norm, decoder_losses[1]/eql_norm)
    print("decoder loss ratio: %f, decoder EQL loss  ratio: %f" % loss_ratios)
    return validation_loss_vals


def create_feed_dictionary(data, params, idxs=None):
    """
    Create the feed dictionary for passing into tensorflow.

    Arguments:
        data - Dictionary object containing the data to be passed in. Must contain input data x,
        along the first (and possibly second) order time derivatives dx (ddx).
        params - Dictionary object containing model and training parameters. The relevant
        parameters are model_order (which determines whether the SINDy model predicts first or
        second order time derivatives), sequential_thresholding (which indicates whether or not
        coefficient thresholding is performed), coefficient_mask (optional if sequential
        thresholding is performed; 0/1 mask that selects the relevant coefficients in the SINDy
        model), and learning rate (float that determines the learning rate).
        idxs - Optional array of indices that selects which examples from the dataset are passed
        in to tensorflow. If None, all examples are used.

    Returns:
        feed_dict - Dictionary object containing the relevant data to pass to tensorflow.
    """
    if idxs is None:
        idxs = np.arange(data['x'].shape[0])
    feed_dict = {}
    feed_dict['x:0'] = data['x'][idxs]
    feed_dict['dx:0'] = data['dx'][idxs]
    if params['model_order'] == 2:
        feed_dict['ddx:0'] = data['ddx'][idxs]
    feed_dict['learning_rate:0'] = params['learning_rate']
    return feed_dict

In [ ]:
num_experiments = 1
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['save_name'] = 'lorenz_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    tf.compat.v1.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)
    df = df.append({**results_dict, **params}, ignore_index=True)

#df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0
Tensor("add_2:0", shape=(None, 3), dtype=float32)
Tensor("add_2:0", shape=(None, 3), dtype=float32)
Starting benchmark for function:	Lorenz_x


ValueError: ignored

In [ ]:
#visualizing data-- let's run the first entry of the validation set through the
#now-trained autoencoder, then do something with the results... how will we do
#this?? i have no idea lmfao

#lowdim = validation_data["s"]

rawdata = results_dict["autoencoder"]
#print(rawdata)

#nn_dz = rawdata["dz_predict"]
#decoder_dz = rawdata["dz"]
#decoder_z = rawdata["z"]

print(params["save_name"])
#print(params["data_path"])


#a = tf.constant([[1, 2], [3, 4]])                 
#b = tf.add(nn_dz, 1)
#out = tf.multiply(nn_dz, b)
#print(nn_dz)
#print(b)
#print(out.eval(session=tf.compat.v1.Session()))   

#print(tf.reduce_mean(nn_dz-decoder_dz))


#with tf.compat.v1.Session() as sess:
#  sess.run(tf.compat.v1.global_variables_initializer())
  


#fig_val = plt.figure()
#ax_val = fig_val.add_subplot(111, projection='3d')
#ax_val.plot(lowdim[0,:,0], lowdim[0,:,1], lowdim[0,:,2], "b--")

#origin = lowdim[0,0]
#transformed = [origin]

#for i in range(len(lowdim)):
#  a = transformed[i] + 0.01*deriv[i]
#  ax_val.plot(a[0], a[1], a[2], "r--")
#  transformed.append(a)

#plt.xticks([])
#plt.axis('off')
#ax_val.view_init(azim=120)



#fig_valp = plt.figure()
#ax_valp = fig_valp.add_subplot(111, projection='3d')
#ax_valp.plot(lowdim_predict[:,0], lowdim_predict[:,1], lowdim_predict[:,2], "r--")


In [ ]:
data_path = os.getcwd() + '/'
#save_name = 'model1'
params = pickle.load(open("/content/lorenz_2021_01_29_01_05_20_133196_params.pkl", 'rb'))
params['save_name'] = data_path + params["save_name"]

print(params["save_name"])

autoencoder_network = rawdata

learning_rate = tf.compat.v1.placeholder(tf.float32, name='learning_rate')
saver = tf.compat.v1.train.Saver(var_list=tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.GLOBAL_VARIABLES))

tensorflow_run_tuple = ()
for key in autoencoder_network.keys():
    tensorflow_run_tuple += (autoencoder_network[key],)

In [ ]:
#yay

t = np.arange(0,20,.01)
z0 = np.array([[-8,7,27]])

test_data = generate_lorenz(z0, t, params['input_dim'], linear=False, normalization=np.array([1/40,1/40,1/40]))

test_data['s'] = test_data['s'].reshape((-1,params['latent_dim'])) 
test_data['ds'] = test_data['ds'].reshape((-1,params['latent_dim']))
test_data['x'] = test_data['x'].reshape((-1,params['input_dim'])) #gotta reshape the x,dx as well to map to dict
test_data['dx'] = test_data['dx'].reshape((-1,params['input_dim']))

with tf.compat.v1.Session() as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
    saver.restore(sess, params["save_name"])
    test_dictionary = create_feed_dictionary(test_data, params)
    tf_results = sess.run(tensorflow_run_tuple, feed_dict=test_dictionary)

test_set_results = {}
for i,key in enumerate(autoencoder_network.keys()):
    test_set_results[key] = tf_results[i]

In [ ]:
from scipy.optimize import fsolve
import math


final_z = test_set_results["z"]
final_dz = test_set_results["dz"]

fig_val = plt.figure(figsize=(10,10),facecolor='w')
ax_val = fig_val.add_subplot(111, projection='3d')

ax_val.plot(final_dz[:,0], final_dz[:,1], final_dz[:,2], "r--")

predictions = test_set_results["dz_predict"]

a = np.array([final_z[0]])

for i in range(len(predictions)):
  [l, m, n] = predictions[i]

  def lorenz_1(vars,du=l, dv=m, dw=n):
    u, v, w = vars
    eq1 = -10*(u - v) - du
    eq2 = 8/3 *u - v - u*w - dv
    eq3 = -28*w + u*v - dw
    return eq1, eq2, eq3

  u,v,w = np.array(fsolve(lorenz_1, tuple(final_z[i])))

  q = np.array([u,v,w])
  a = np.append(a, q).reshape(i+2, 3)


# ax_val.plot(a[:,0], a[:,1], a[:,2], "r--")
#ax_val.plot(predictions[:,0], predictions[:,1], predictions[:,2], "r--")

real_diff = test_data["ds"]

print(predictions)
# ax_val.plot(real_diff[:,0], real_diff[:,1], real_diff[:,2], "b--")
plt.xticks([])
plt.axis('off')
ax_val.view_init(elev=60, azim=90)